In [26]:
import geopandas as gpd
import numpy as np
import pandas as pd
import os

In [2]:
DMA_shp_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\DMA\DMA shape file\DMA.shp"
floods_shp_2023_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\flood\1_extract_flood_inDMA\flood_map\2023.shp"
DMA_mapping_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Anglian_water\processed\DMA_Mapping.csv"

In [3]:
DMA_shp = gpd.read_file(DMA_shp_path)
floods_shp_2023 = gpd.read_file(floods_shp_2023_path)
DMA_shp=DMA_shp.to_crs(epsg = 3857)
floods_shp_2023 = floods_shp_2023.to_crs(epsg=3857)

In [4]:
overlap_2023 = gpd.overlay(DMA_shp,floods_shp_2023,how='intersection',keep_geom_type=False)
overlap_2023 = overlap_2023.drop_duplicates().reset_index(drop = True)

In [5]:
DMA_mapping = pd.read_csv(DMA_mapping_path)

In [6]:
num_DMA = len(overlap_2023)
num_DMA

223

In [7]:
DMA_counts = overlap_2023.groupby('flood_id').size().reset_index(name='dma_count')
DMA_counts

,flood_id,dma_count
0,570,2
1,571,1
2,572,1
3,573,1
4,574,3
...,...,...
155,726,3
156,727,2
157,728,1
158,729,1


In [9]:
dma_num_by_floods = DMA_counts.groupby('dma_count').size().reset_index(name='num_floods')
# dma_num_by_floods['num_dma'] = np.arange(1,max(DMA_counts['dma_count'])+1)

In [10]:
dma_num_by_floods

,dma_count,num_floods
0,1,106
1,2,45
2,3,9


In [11]:
cause_num_by_floods = overlap_2023.groupby('flood_caus').size().reset_index(name='num')

In [12]:
cause_num_by_floods

,flood_caus,num
0,channel capacity exceeded (no raised defences),91
1,operational failure/breach of defence,14
2,other,40
3,overtopping of defences,78


In [13]:
floods_counts = overlap_2023.groupby('DMASHORTNA').size().reset_index(name='floods_count')

In [14]:
floods_counts

,DMASHORTNA,floods_count
0,ALLINGMA,3
1,ASBITMUA,4
2,ASHBHLMA,13
3,ASHBWTMA,3
4,BBBARDMA,6
5,BILLHYMA,12
6,BLANKNMA,1
7,BRANDNMA,1
8,BRANTRMA,1
9,BRAYFOMA,1


In [15]:
floods_num_by_date = floods_shp_2023.groupby(['start_date', 'end_date']).size().reset_index(name='counts_date')
floods_num_by_date

,start_date,end_date,counts_date
0,2023-10-20,2023-10-23,160


In [17]:
dma_per_flood = overlap_2023.groupby('flood_id')['DMASHORTNA'].apply(list).reset_index(name='DMA_ids')
dma_per_flood

,flood_id,DMA_ids
0,570,"[WILSTWMA, INGSTOMA]"
1,571,[FISKERMA]
2,572,[SCAMBLMA]
3,573,[CONINGMA]
4,574,"[FALDINMA, LANGWBMA, LANBARMA]"
...,...,...
155,726,"[BRANTRMA, NHYKEVMA, MILLMOMA]"
156,727,"[BSSHAMMA, WADHARMA]"
157,728,[HORNNWMA]
158,729,[LEADENMA]


In [18]:
dma_name_to_id = dict(zip(DMA_mapping['DMA_Name'], DMA_mapping['DMA_ID']))
dma_per_flood['DMA_ids'] = dma_per_flood['DMA_ids'].apply(
    lambda x: [dma_name_to_id[dma] for dma in x if dma in dma_name_to_id]
)
flood_shp_2023 = floods_shp_2023.merge(dma_per_flood,on='flood_id',how='left')

In [29]:
flood_shp_2023.drop(columns = ['rec_out_id','rec_grp_id','fm_status','hfm_status','data_prov','data_qual','year'],inplace = True)


KeyError: "['rec_out_id', 'rec_grp_id', 'fm_status', 'hfm_status', 'data_prov', 'data_qual', 'year'] not found in axis"

In [22]:
flood_shp_2023

,name,start_date,end_date,flood_src,flood_caus,data_src,fluvial_f,coastal_f,tidal_f,flood_id,geometry,DMA_ids
0,LNA_2023_10_Fluvial_River Till,2023-10-20,2023-10-23,main river,channel capacity exceeded (no raised defences),Satellite - Radar,True,False,False,570,"MULTIPOLYGON Z (((-71262.002 7041169.011 0, -7...","[1949, 877]"
1,LNA_2023_10_Fluvial_Barlings Eau,2023-10-20,2023-10-23,main river,overtopping of defences,Aerial Photography,True,False,False,571,"MULTIPOLYGON Z (((-37520.51 7025983.524 0, -37...",[611]
2,LNA_2023_10_Fluvial_Upper River Bain,2023-10-20,2023-10-23,main river,channel capacity exceeded (no raised defences),Aerial Photography,True,False,False,572,"POLYGON Z ((-15955.309 7028666.372 0, -15971.3...",[1495]
3,LNA_2023_10_Fluvial_Lower River Bain,2023-10-20,2023-10-23,main river,operational failure/breach of defence,Aerial Photography,True,False,False,573,"POLYGON Z ((-18497.508 7003516.038 0, -18492.0...",[422]
4,LNA_2023_10_Fluvial_Barlings Eau,2023-10-20,2023-10-23,main river,channel capacity exceeded (no raised defences),Aerial Photography,True,False,False,574,"MULTIPOLYGON Z (((-46330.063 7035740.195 0, -4...","[592, 1014, 1007]"
...,...,...,...,...,...,...,...,...,...,...,...,...
155,LNA_2023_10_Fluvial_Upper River Witham,2023-10-20,2023-10-23,ordinary watercourse,other,Satellite - Radar,True,False,False,726,"MULTIPOLYGON Z (((-63484.498 7015947.774 0, -6...","[230, 1222, 1130]"
156,LNA_2023_10_Fluvial_River Brant,2023-10-20,2023-10-23,ordinary watercourse,other,Satellite - Radar,True,False,False,727,"MULTIPOLYGON Z (((-66061.511 7003556.891 0, -6...","[290, 1853]"
157,LNA_2023_10_Fluvial_Old River Bain,2023-10-20,2023-10-23,main river,channel capacity exceeded (no raised defences),Other,True,False,False,728,"POLYGON Z ((-13447.319 7021317.884 0, -13428.7...",[846]
158,LNA_2023_10_Fluvial_River Brant,2023-10-20,2023-10-23,main river,channel capacity exceeded (no raised defences),Aerial Photography,True,False,False,729,"MULTIPOLYGON Z (((-68276.674 6997280.63 0, -68...",[1021]


In [23]:
output_dir = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\flood\2_extract_flood_date_waterflow\2023"
water_flow_path = r"D:\qing_research\1_flood_water\1_flood_water_network\data\Anglian_water\processed\DMA_flow"

In [24]:
flood_shp_2023['start_date'] = pd.to_datetime(flood_shp_2023['start_date'])
flood_shp_2023['end_date'] = pd.to_datetime(flood_shp_2023['end_date'])

In [27]:
for _, flood_row in flood_shp_2023.iterrows():
    dma_ids = flood_row['DMA_ids']
    for dma_id in dma_ids:
        dma_file = f"Flow_DMA_{dma_id}.csv"
        dma_file_path = os.path.join(water_flow_path,dma_file)
        dma_data = pd.read_csv(dma_file_path)
        dma_data['Date_Time'] = pd.to_datetime(dma_data['Date_Time'])
        
        expanded_startdate = flood_row['start_date'] - pd.Timedelta(days = 7)
        expanded_enddate = flood_row['end_date'] + pd.Timedelta(days = 7)
        extracted_data = dma_data[(dma_data['Date_Time']>=expanded_startdate) & (dma_data['Date_Time']<=expanded_enddate)]

        output_file_name = f"flood_{flood_row['flood_id']}_{flood_row['start_date'].strftime('%Y%m%d')}_{flood_row['end_date'].strftime('%Y%m%d')}_{dma_id}.csv"
        output_path = os.path.join(output_dir,output_file_name)
        extracted_data.to_csv(output_path,index=False)

In [31]:
flood_shp_2023['start_date'] = flood_shp_2023['start_date'].dt.date
flood_shp_2023['end_date'] = flood_shp_2023['end_date'].dt.date

In [32]:
flood_shp_2023.to_file(fr"D:\qing_research\1_flood_water\1_flood_water_network\data\Geo\flood\2_extract_flood_date_waterflow\2023\flood_shp_2023.shp", driver = "ESRI Shapefile")